Introduction

Import statements

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

Load the 2020 data set and the reference files.

In [2]:
a = pd.read_csv("data/2020/2020data.csv")
b = pd.read_csv('data/2020/2020datasimp.csv')
c = pd.read_csv('data/2020/map-keys-2020.csv')

print(a.columns)
print(b.columns)
print(c.columns)

Index(['id', 'obs', 'place', 'date', 'time', 'sci', 'name', 'notes'], dtype='object')
Index(['place', 'date', 'time', 'sci', 'name'], dtype='object')
Index(['X', 'Y', 'Name', 'description'], dtype='object')


In [3]:
c.rename(columns={'X':'lon', 'Y':'lat', 'Name':'place'}, inplace=True)

In [4]:
a.iloc[:5]

,id,obs,place,date,time,sci,name,notes
0,02777507-6356-5477-92eb-6d7aea343265,NaN,alleestrasse 1,2020-09-02,11:09:04 AM,plantago lanceolata,plantain lancéolé,NaN
1,fd359c20-af7c-5560-b412-56bfbbd154e3,NaN,alleestrasse 1,2020-09-02,11:08:19 AM,centaurea nigra,centaurée noire,could be jaucee
2,469cc29c-f377-5312-ba11-f222ca888cec,NaN,alleestrasse 1,2020-09-02,11:06:59 AM,plantago media,plantain moyen,NaN
3,c7613f5f-b22d-5b51-832f-06cb61bc4128,NaN,alleestrasse 1,2020-09-02,11:06:05 AM,chenopodium album agg.,chénopode blanc,NaN
4,9bba2690-f7a5-54b4-bd74-74fd056ccb3a,NaN,alleestrasse 1,2020-09-02,11:05:18 AM,centaurea jacea agg.,centaurée jacée,NaN


In [5]:
b.iloc[:5]

,place,date,time,sci,name
0,alleestrasse 1,2020-09-02,11:09:04 AM,plantago lanceolata,plantain lancéolé
1,alleestrasse 1,2020-09-02,11:08:19 AM,centaurea nigra,centaurée noire
2,alleestrasse 1,2020-09-02,11:06:59 AM,plantago media,plantain moyen
3,alleestrasse 1,2020-09-02,11:06:05 AM,chenopodium album agg.,chénopode blanc
4,alleestrasse 1,2020-09-02,11:05:18 AM,centaurea jacea agg.,centaurée jacée


In [6]:
c.iloc[:5]

,lon,lat,place,description
0,7.253285,47.138823,Jura 2,5.06
1,7.252867,47.137585,Salome 3,NaN
2,7.252760,47.137273,Salome 4,NaN
3,7.252618,47.136619,Alleestrasse 4\n,NaN
4,7.261210,47.143164,Energie 1,NaN


In [7]:
# the place variable does not match in the data frames
# df['text'].str.split('::').str[0]
c['place'] = c.place.str.split('\n').str[0]

# get rid of uppercase
c.place = c.place.str.lower()
b.place = b.place.str.lower()

# get rid of spaces
c.place = c.place.map(lambda x:  x.replace(" ","-"))
b.place = b.place.map(lambda x:  x.replace(" ","-"))


In [8]:
# find place names that don't match:
a_locs = a.place.unique()
b_locs = b.place.unique()
c_locs = c.place.unique()

# get the symetric difference between the different place name lists
placenames = list(set(c_locs) | set(b_locs))
intersect_placenames = list(set(c_locs) & set(b_locs))
print(F"There are {len(placenames)} different place names\n")
print(F"\nThere are {len(b_locs)} different names in b\n")
print(F"\nThere are {len(c_locs)} different names in c\n")
print(F"\nThere are {len(intersect_placenames)} shared between b and c")

There are 121 different place names


There are 115 different names in b


There are 109 different names in c


There are 103 shared between b and c


In [9]:
print(F"The shared place names:\n\n{intersect_placenames}")

The shared place names:

['salome-3', 'truite-3', 'swatch-1', 'truite-5', 'chapelle', 'parc-muni', 'boujean-1', 'cygnes-lot-8', 'energie-1', 'energie-2', 'pery-taubenlochweg', 'weissenau', 'lucherz-bureli', 'cygnes', 'frinvillier-fabrique-4', 'schlosslistrasse', 'cygnes-lot-6', 'cygnes-lot-3', 'schlosslifeld-3', 'sundgraben-forest-left', 'orvin-petit-moulin', 'gottstatt-3', 'college', 'frinvillier-fabrique-1', 'schussinsel-2', 'schussinsel-1', 'salome-5', 'reichenbach', 'lezard-1', 'cygnes-lot-4', 'sonnenfeld-1', 'cygnes-lot-7', 'cygnes-lot-2', 'frinvilier-fabrique-3', 'salome-4', 'cygnes-lot-1', 'saint-ursanne-falls', 'cff-path-2', 'gottstatt-1', 'schussinsel-4', 'cheyres-reserve', 'frinvillier-ramp', 'taubenloch-2', 'buren-1', 'bluets-2', 'truite-2', 'buren-3', 'chemin-des-voies', 'energie-3', 'transjurane-tunnel', 'alleestrasse-2', 'jura-1', 'salome-1', 'sureaux', 'bluets-1', 'cheyres-path', 'salome-2', 'schussinsel-5', 'sundgraben-forest-right', 'football-6', 'cff-lot-2', 'boujean-

In [10]:
# places in b that do not mathc c:
b_no_match = [x for x in b_locs if x not in c_locs]
b_no_match

['frinvilier-fabrique-1',
 'frinvilier-fabrique-4',
 'frinvillier-fabrique-2',
 'rue-de-l’eau-1',
 'rue-de-l’eau-2',
 'rue-de-l’eau-3',
 'rue-de-l’eau-4',
 'rue-de-l’eau-5',
 'schusinsel-3',
 'sundgraben-bridge-left',
 'sundgraben-port',
 'twann-gorges']

Clean 2020 data set and reference files

In [11]:
data2020 = utility.removelistreturns(data2020)
floralist = utility.removelistreturns(floralist)
data2020 = utility.makenestedlist(data2020)
floralist = utility.makenestedlist(floralist)
data2020 = utility.arrange2020data(data2020)
floralist = utility.arrangeflorahelveticadata(floralist)
data2020[3] = utility.removenameendings(data2020[3])
data2020 = utility.fixscinames(data2020)
data2020 = utility.fixdate(data2020)
data2020 = utility.fixtime(data2020)

NameError: name 'utility' is not defined

Initial data analysis

In [ ]:
#total number of different values for each variable in data.
occurencesummary = utility.occurencelists(data2020)
print(occurencesummary[3])
newfile = open("occurencesummary.txt", 'w')
newfile.write(occurencesummary)
newfile.close()
comparison = utility.compared(occurencesummary[3][1], floralist[9])
print(comparison)